In [ ]:
import sys
import os

sys.path.append('F:/ai-ml/itapia/backend/evo_worker')
sys.path.append('F:/ai-ml/itapia/backend/shared')

os.environ['POSTGRES_HOST'] = 'localhost'
os.environ['REDIS_HOST'] = 'localhost'

In [ ]:
import app.core.config as cfg 
cfg.PARALLEL_MULTICONTEXT_LIMIT = 12

In [ ]:
from app.dependencies import create_dependencies, get_backtest_context_manager
create_dependencies()

context_mng = get_backtest_context_manager()
context_mng.init_ready_contexts()

In [ ]:
import nest_asyncio
nest_asyncio.apply()

for _, context in context_mng.contexts.items():
    if context.ticker in cfg.FOR_TEST_CONTEXTS:
        await context.load_data_into_memory()

In [ ]:
len(context_mng.get_ready_contexts())

In [ ]:
from app.backtest.action import MEDIUM_SWING_IDEAL_MAPPER, MEDIUM_SWING_PESSIMISTIC_MAPPER
from app.backtest.evaluator import MultiContextEvaluator

evaluator = MultiContextEvaluator(contexts=context_mng.get_ready_contexts(),
                                         aggregation_method='trim-mean')

evaluator.add_evaluate_scenario(MEDIUM_SWING_IDEAL_MAPPER, weight=0.3)
evaluator.add_evaluate_scenario(MEDIUM_SWING_PESSIMISTIC_MAPPER, weight=0.7)

In [ ]:
from itapia_common.rules.nodes.registry import get_operators_by_type, get_terminals_by_type
operators_by_type = get_operators_by_type()
terminals_by_type = get_terminals_by_type()

In [ ]:
from app.algorithms.pop import DominanceIndividual

In [ ]:
from app.algorithms.operators.construct import RandomMaxDepthInitOperator
from itapia_common.schemas.entities.rules import SemanticType
init_opr = RandomMaxDepthInitOperator(purpose=SemanticType.DECISION_SIGNAL, max_depth=10,
                                      ind_cls=DominanceIndividual,
                                      terminals_by_type=terminals_by_type,
                                      operators_by_type=operators_by_type)

In [ ]:
from app.algorithms.operators.crossover import SubtreeCrossoverOperator
crossover_opr = SubtreeCrossoverOperator()

from app.algorithms.operators.mutation import SubtreeMutationOperator
mutation_opr = SubtreeMutationOperator(max_subtree_depth=4,
                                       terminals_by_type=terminals_by_type,
                                       operators_by_type=operators_by_type)

from app.algorithms.operators.selection import TournamentSelectionOperator
from app.algorithms.comparator import RankAndCrowdingComparator, FixedDominateComparator
selection_opr = TournamentSelectionOperator(k=8, comparator=RankAndCrowdingComparator())

from app.algorithms.operators.replacement import NSGA2ReplacementOperator
replacement_opr = NSGA2ReplacementOperator(comparator=FixedDominateComparator())

from app.algorithms.objective import CSRSPObjectiveExtractor
obj_extractor = CSRSPObjectiveExtractor()

In [ ]:
from app.algorithms.engine.nsga2 import NSGA2EvoEngine
from datetime import datetime
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

config = {
    'pop_size': 100,
    'num_gen': 200,
    'pc': 0.85,
    'pm': 0.25
}

engine = NSGA2EvoEngine(run_id=f'nsga2-{now}-first',
                        evaluator=evaluator,
                        obj_extractor=obj_extractor,
                        init_opr=init_opr,
                        crossover_opr=crossover_opr,
                        mutation_opr=mutation_opr,
                        selection_opr=selection_opr,
                        replacement_opr=replacement_opr,
                        dominate_comparator=FixedDominateComparator(),
                        seeding_rules=None,
                        **config)

In [ ]:
#front = engine.run(stop_gen=8)

In [ ]:
engine.fallback_state['pop']

In [ ]:
from itapia_common.dblib.services import EvoService
from itapia_common.dblib.session import get_rdbms_session
rdbms_session = next(get_rdbms_session())
evo_service = EvoService(rdbms_session)

In [ ]:
engine.run_id

In [ ]:
evo_run_ent = evo_service.get_evo_run('nsga2-2025-09-01_15-39-57-first')

In [ ]:
import pickle


fallback_state = pickle.loads(evo_run_ent.fallback_state)